In [0]:
%sql
use catalog hive_metastore;
use default;

In [0]:
# %sql
#  create table default.earthqu_details(id string, Longitude double, Latitude double, Depth double, title string, place_description string, sig long, mag double, magType string, time timestamp, updated timestamp) 
# Alter table default.earthqu_details add column country_code string, sig_classs string;

org.apache.spark.sql.catalyst.ExtendedAnalysisException: [FIELDS_ALREADY_EXISTS] Cannot add column, because `country_code` already exists in "STRUCT<id: STRING, Longitude: DOUBLE, Latitude: DOUBLE, Depth: DOUBLE, title: STRING, place_description: STRING, sig: BIGINT, mag: DOUBLE, magType: STRING, time: TIMESTAMP, updated: TIMESTAMP, country_code: STRING, sig_classs: STRING>". SQLSTATE: 42710; line 2 pos 0;
AddColumns [QualifiedColType(None,country_code,StringType,true,None,None,None,None), QualifiedColType(None,sig_classs,StringType,true,None,None,None,None)]
+- ResolvedTable com.databricks.sql.managedcatalog.UnityCatalogV2Proxy@272f1b06, default.earthqu_details, DeltaTableV2(org.apache.spark.sql.SparkSession@40bba02f,abfss://unity-catalog-storage@dbstorage6wpicghpifqau.dfs.core.windows.net/285375790323442/__unitystorage/catalogs/a7b3235a-f66f-450d-a09f-dea291e382d1/tables/432b22e2-b4e5-4c71-b87e-47e45221a798,Some(CatalogTable(
Catalog: earthquake_pip
Database: default
Table: earthqu_d

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
import reverse_geocoder as rg

def get_country_code(lat, lang):
  """
  using the reverse_geocoder library, get the country code for a given lat, lang

  Parameters:
  - lat: latitude coordinate (double)
  - lang: longitude coordinate (double)

  returns:
  - country code (string)

  ex: get_country_code(40.7128, -74.0060) -> 'US'
  """
  try:
    coords = [float(lat), float(lang)]
    result = rg.search([lat, lang])[0].get("cc")
    return result
  except Exception as e:
    print("can't get the country code of lat: {lat}, lang: {lang}, error: {e}")
    return None

get_country_code_udf = F.udf(get_country_code, T.StringType())


# glopal variables
storage_account = "ataaearthquakelake"
datalake_key = "xxxxxxxxx"
bucket_name = "silver"
folder_path = "earthqu_details_silver"
silver_data_location = f"abfss://{bucket_name}@{storage_account}.dfs.core.windows.net/{folder_path}/"
print(silver_data_location)


spark.conf.set(f"fs.azure.account.key.{storage_account}.dfs.core.windows.net", 
               datalake_key)

abfss://silver@ataaearthquakelake.dfs.core.windows.net/earthqu_details_silver/


In [0]:
max_date_query = """select max(creation_date) as max_date  from delta.`abfss://gold@ataaearthquakelake.dfs.core.windows.net/earthqu_details_gold`"""

max_date = spark.sql(max_date_query).collect()[0][0]

df_silver = spark.read.format("parquet").load(silver_data_location).filter(F.col("creation_date") > max_date)

df_silver_witc_cc = df_silver.withColumn("country_code", get_country_code_udf(F.col("Latitude"), F.col("Longitude")))

df_gold = df_silver_witc_cc.withColumn("sig_classs",
                              F.when(F.col("sig") < 100, "nano").\
                              when( (F.col("sig") >= 100) & (F.col("sig") < 500),  "micro" ).otherwise("macro"))

if not df_gold.isEmpty():
    bucket_name = "gold"
    folder_path = "earthqu_details_gold"
    gold_data_location = f"abfss://{bucket_name}@{storage_account}.dfs.core.windows.net/{folder_path}/"
    df_gold.write\
                  .format("delta")\
                  .mode("append")\
                  .save(gold_data_location)
    df_gold.write.mode("append").saveAsTable("earthqu_details")
    
elif df_gold.isEmpty():
    print("no new data to write")


no new data to write


In [0]:
%sql
select * from default.earthqu_details; 

id,Longitude,Latitude,Depth,title,place_description,sig,mag,magType,time,updated,country_code,sig_classs,creation_date
nc75131932,-122.700836181641,38.7723350524902,1.71000003814697,"M 1.5 - 1 km W of Anderson Springs, CA","1 km W of Anderson Springs, CA",36,1.53,md,2025-02-11T23:59:34.93Z,2025-02-12T00:01:11.676Z,US,nano,2025-02-12
tx2025cytc,-103.726,31.109,7.1903,"M 1.6 - 13 km N of Balmorhea, Texas","13 km N of Balmorhea, Texas",39,1.6,ml,2025-02-11T23:58:35.551Z,2025-02-12T12:51:32.464Z,US,nano,2025-02-12
av93496991,-166.783666666667,53.8566666666667,8.69,"M -0.2 - 16 km WSW of Dutch Harbor, Alaska","16 km WSW of Dutch Harbor, Alaska",0,-0.22,ml,2025-02-11T23:56:17.55Z,2025-02-12T03:09:49.72Z,US,nano,2025-02-12
us7000pd19,-97.603,38.6739,9.85,"M 2.6 - 0 km S of Assaria, Kansas","0 km S of Assaria, Kansas",104,2.6,mb_lg,2025-02-11T23:53:56.735Z,2025-02-12T10:56:22.848Z,US,micro,2025-02-12
ak0251xujfzd,-146.5587,61.456,19.7,"M 1.8 - 37 km NNW of Valdez, Alaska","37 km NNW of Valdez, Alaska",50,1.8,ml,2025-02-11T23:46:18.01Z,2025-02-11T23:48:15.235Z,US,nano,2025-02-12
ak0251xugg1c,-142.7479,60.46,1.2,"M 2.1 - 108 km S of McCarthy, Alaska","108 km S of McCarthy, Alaska",68,2.1,ml,2025-02-11T23:32:19.044Z,2025-02-11T23:34:18.108Z,US,nano,2025-02-12
nc75131917,-122.878,38.8461666666667,3.08,"M 0.3 - 13 km E of Cloverdale, CA","13 km E of Cloverdale, CA",1,0.29,md,2025-02-11T23:25:54.72Z,2025-02-12T09:02:19.67Z,US,nano,2025-02-12
uu80102086,-113.3655,37.9588333333333,-0.02,"M 1.9 - 36 km NNE of Newcastle, Utah","36 km NNE of Newcastle, Utah",54,1.87,md,2025-02-11T23:19:57.68Z,2025-02-11T23:34:34.78Z,US,nano,2025-02-12
us7000pd15,121.0709,18.5379,39.604,"M 4.6 - 7 km S of Claveria, Philippines","7 km S of Claveria, Philippines",326,4.6,mb,2025-02-11T23:18:15.941Z,2025-02-12T04:37:40.04Z,PH,micro,2025-02-12
av93032619,-135.755666666667,57.1038333333333,5.54,"M -0.4 - 26 km WNW of Sitka, Alaska","26 km WNW of Sitka, Alaska",0,-0.43,ml,2025-02-11T23:14:58.13Z,2025-02-12T03:22:03.34Z,US,nano,2025-02-12
